# Plotly Analysis on The Filtered 311 Data

### Libraries

In [1]:
import modin.pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt
from IPython.display import display

import plotly.plotly as py # interactive graphing
from plotly.graph_objs import Bar, Scatter, Marker, Layout
import plotly.tools as tls

import chart_studio
chart_studio.tools.set_credentials_file(username='Alex23_shi', api_key='cVYhS4Y50yt8I8DuyZr6')

Dask needs bokeh >= 0.13.0 for the dashboard.
Continuing without the dashboard.


### Prepare the data

In [2]:
display(pd.read_csv('data/311_filtered1.csv',nrows=2).head())
display(pd.read_csv('data/311_filtered1.csv',nrows=2).tail())

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

Parameters provided defaulting to pandas implementation.
To request implementation, send an email to feature_requests@modin.org.



Created Date             Closed Date Agency     Complaint Type  \
0  07/29/2011 07:51:03 PM  08/29/2011 08:43:43 PM    DPR       Damaged Tree   
1  07/29/2011 08:34:33 PM  07/30/2011 05:38:24 AM    DOT  Broken Muni Meter   

                       Descriptor Location Type      City   Latitude  \
0  Branch or Limb Has Fallen Down        Street   JAMAICA  40.720235   
1                      No Receipt        Street  NEW YORK  40.759540   

   Longitude  
0 -73.789196  
1 -73.972245

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

Parameters provided defaulting to pandas implementation.



Created Date             Closed Date Agency     Complaint Type  \
0  07/29/2011 07:51:03 PM  08/29/2011 08:43:43 PM    DPR       Damaged Tree   
1  07/29/2011 08:34:33 PM  07/30/2011 05:38:24 AM    DOT  Broken Muni Meter   

                       Descriptor Location Type      City   Latitude  \
0  Branch or Limb Has Fallen Down        Street   JAMAICA  40.720235   
1                      No Receipt        Street  NEW YORK  40.759540   

   Longitude  
0 -73.789196  
1 -73.972245

In [3]:
disk_engine = create_engine('sqlite:///311.db')
start = dt.datetime.now()
chunksize = 50000
j = 0
index_start = 1

for df in pd.read_csv('data/311_filtered1.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Remove the un-interesting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'CreatedDate', 'ClosedDate', 'TimeToCompletion',
               'City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1
    print ('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

Parameters provided defaulting to pandas implementation.

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\pandas\datetimes.py:56: FutureWarning:

the 'box' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'box'



14 seconds: completed 50000 rows


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`to_sql` defaulting to pandas implementation.



29 seconds: completed 100000 rows
44 seconds: completed 150000 rows
58 seconds: completed 200000 rows
73 seconds: completed 250000 rows
87 seconds: completed 300000 rows
102 seconds: completed 350000 rows
116 seconds: completed 400000 rows
130 seconds: completed 450000 rows
144 seconds: completed 500000 rows
158 seconds: completed 550000 rows
172 seconds: completed 600000 rows
186 seconds: completed 650000 rows
200 seconds: completed 700000 rows
214 seconds: completed 750000 rows
228 seconds: completed 800000 rows
242 seconds: completed 850000 rows
255 seconds: completed 900000 rows
269 seconds: completed 950000 rows
283 seconds: completed 1000000 rows
297 seconds: completed 1050000 rows
311 seconds: completed 1100000 rows
325 seconds: completed 1150000 rows
339 seconds: completed 1200000 rows
353 seconds: completed 1250000 rows
366 seconds: completed 1300000 rows
380 seconds: completed 1350000 rows
394 seconds: completed 1400000 rows
408 seconds: completed 1450000 rows
422 seconds: co

2492 seconds: completed 11300000 rows
2503 seconds: completed 11350000 rows
2514 seconds: completed 11400000 rows
2525 seconds: completed 11450000 rows
2540 seconds: completed 11500000 rows
2554 seconds: completed 11550000 rows
2568 seconds: completed 11600000 rows
2582 seconds: completed 11650000 rows
2596 seconds: completed 11700000 rows
2610 seconds: completed 11750000 rows
2624 seconds: completed 11800000 rows
2638 seconds: completed 11850000 rows
2649 seconds: completed 11900000 rows
2660 seconds: completed 11950000 rows
2671 seconds: completed 12000000 rows
2682 seconds: completed 12050000 rows
2693 seconds: completed 12100000 rows
2704 seconds: completed 12150000 rows
2715 seconds: completed 12200000 rows
2726 seconds: completed 12250000 rows
2737 seconds: completed 12300000 rows
2748 seconds: completed 12350000 rows
2759 seconds: completed 12400000 rows
2770 seconds: completed 12450000 rows
2781 seconds: completed 12500000 rows
2792 seconds: completed 12550000 rows
2803 seconds

In [4]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', disk_engine)
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



index                 CreatedDate                  ClosedDate Agency  \
0      1  2011-07-29 19:51:03.000000  2011-08-29 20:43:43.000000    DPR   
1      2  2011-07-29 20:34:33.000000  2011-07-30 05:38:24.000000    DOT   
2      3  2011-07-29 22:20:18.000000  2011-07-29 23:17:35.000000   NYPD   

         ComplaintType                      Descriptor      City  
0         Damaged Tree  Branch or Limb Has Fallen Down   JAMAICA  
1    Broken Muni Meter                      No Receipt  NEW YORK  
2  Noise - Residential                    Loud Talking  BROOKLYN

In [5]:
df = pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', disk_engine)
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



Agency                      Descriptor
0    DPR  Branch or Limb Has Fallen Down
1    DOT                      No Receipt
2   NYPD                    Loud Talking

In [6]:
df = pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', disk_engine)
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



Agency                      Descriptor
0    DPR  Branch or Limb Has Fallen Down
1    DOT                      No Receipt
2   NYPD                    Loud Talking

In [7]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'LIMIT 10', disk_engine)
df

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



ComplaintType                      Descriptor Agency
0             Damaged Tree  Branch or Limb Has Fallen Down    DPR
1        Broken Muni Meter                      No Receipt    DOT
2      Noise - Residential                    Loud Talking   NYPD
3      Noise - Residential                Loud Music/Party   NYPD
4       Noise - Commercial                Loud Music/Party   NYPD
5  Noise - Street/Sidewalk                Loud Music/Party   NYPD
6         Blocked Driveway                  Partial Access   NYPD
7       Noise - Commercial                Loud Music/Party   NYPD
8       Noise - Commercial                Loud Music/Party   NYPD
9  Noise - Street/Sidewalk                Loud Music/Party   NYPD

In [8]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency = "NYPD" '
                       'LIMIT 10', disk_engine)
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



ComplaintType        Descriptor Agency
0      Noise - Residential      Loud Talking   NYPD
1      Noise - Residential  Loud Music/Party   NYPD
2       Noise - Commercial  Loud Music/Party   NYPD
3  Noise - Street/Sidewalk  Loud Music/Party   NYPD
4         Blocked Driveway    Partial Access   NYPD

In [9]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB")'
                       'LIMIT 10', disk_engine)
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



ComplaintType        Descriptor Agency
0      Noise - Residential      Loud Talking   NYPD
1      Noise - Residential  Loud Music/Party   NYPD
2       Noise - Commercial  Loud Music/Party   NYPD
3  Noise - Street/Sidewalk  Loud Music/Party   NYPD
4         Blocked Driveway    Partial Access   NYPD

### Plotting Starts Here

In [10]:
df = pd.read_sql_query('SELECT Agency, COUNT(*) as `num_complaints`'
                       'FROM data '
                       'GROUP BY Agency '
                       'ORDER BY -num_complaints', disk_engine)

py.iplot([Bar(x=df.Agency, y=df.num_complaints)], filename='Most common complaints by agency')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



In [11]:
df = pd.read_sql_query('SELECT City, COUNT(*) as `num_complaints` '
                        'FROM data '
                        'GROUP BY `City` '
                       'COLLATE NOCASE '
                       'ORDER BY -num_complaints '
                       'LIMIT 11 ', disk_engine)
df

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



City  num_complaints
0        BROOKLYN         9807411
1           BRONX         6263540
2        NEW YORK         5826867
3   STATEN ISLAND         1307769
4         JAMAICA          710658
5        FLUSHING          515180
6         ASTORIA          483538
7       RIDGEWOOD          393535
8          CORONA          254197
9        WOODSIDE          229604
10       ELMHURST          228532

In [12]:
cities = list(df.City)
#cities.remove(None)

In [13]:
traces = [] # the series in the graph - one trace for each city

for city in cities:
    df = pd.read_sql_query('SELECT ComplaintType, COUNT(*) as `num_complaints` '
                           'FROM data '
                           'WHERE City = "{}" COLLATE NOCASE '
                           'GROUP BY `ComplaintType` '
                           'ORDER BY -num_complaints'.format(city), disk_engine)

    traces.append(Bar(x=df['ComplaintType'], y=df.num_complaints, name=city.capitalize()))

py.iplot({'data': traces, 'layout': Layout(barmode='stack', xaxis={'tickangle': 40}, margin={'b': 150})}, filename='complaints by city stacked')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



In [14]:
#py.iplot({'data': traces, 'layout': Layout(barmode='stack', xaxis={'tickangle': 40}, margin={'b': 150})}, title="Most complaints by city stacked",filename='complaints by city stacked')

In [15]:
for trace in traces:
    trace['y'] = 100.*trace['y']/sum(trace['y'])

In [16]:
py.iplot({'data': traces, 
          'layout': Layout(
                barmode='group',
                xaxis={'tickangle': 40, 'autorange': False, 'range': [-0.5, 16]},
                yaxis={'title': 'Percent of Complaints by City'},
                margin={'b': 150},
                title='Relative Number of 311 Complaints by City')
         }, filename='relative complaints by city')

In [17]:
df = pd.read_sql_query('SELECT ComplaintType, CreatedDate, City '
                       'FROM data '
                       'WHERE CreatedDate < "2014-11-16 23:47:00" '
                       'AND CreatedDate > "2014-11-16 23:45:00"', disk_engine)

df

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



ComplaintType                 CreatedDate       City
0        Noise - Commercial  2014-11-16 23:46:08.000000    ASTORIA
1   Noise - Street/Sidewalk  2014-11-16 23:45:10.000000   NEW YORK
2          Blocked Driveway  2014-11-16 23:45:43.000000   BROOKLYN
3          Derelict Vehicle  2014-11-16 23:46:57.000000  RIDGEWOOD
4        School Maintenance  2014-11-16 23:45:41.000000      BRONX
5         Derelict Vehicles  2014-11-16 23:46:00.000000    Jamaica
6        Noise - Commercial  2014-11-16 23:46:08.000000    ASTORIA
7   Noise - Street/Sidewalk  2014-11-16 23:45:10.000000   NEW YORK
8          Blocked Driveway  2014-11-16 23:45:43.000000   BROOKLYN
9          Derelict Vehicle  2014-11-16 23:46:57.000000  RIDGEWOOD
10       School Maintenance  2014-11-16 23:45:41.000000      BRONX
11        Derelict Vehicles  2014-11-16 23:46:00.000000    Jamaica

In [18]:
df = pd.read_sql_query('SELECT CreatedDate, '
                              'strftime(\'%H\', CreatedDate) as hour, '
                              'ComplaintType '
                       'FROM data '
                       'LIMIT 5 ', disk_engine)
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



CreatedDate hour        ComplaintType
0  2011-07-29 19:51:03.000000   19         Damaged Tree
1  2011-07-29 20:34:33.000000   20    Broken Muni Meter
2  2011-07-29 22:20:18.000000   22  Noise - Residential
3  2011-07-29 22:35:13.000000   22  Noise - Residential
4  2011-07-29 23:46:33.000000   23   Noise - Commercial

In [19]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as hour,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'GROUP BY hour', disk_engine)

df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



CreatedDate hour  Complaints per Hour
0  2011-07-30 00:59:25.000000   00              7891823
1  2011-07-30 01:46:51.000000   01               641433
2  2011-07-30 02:18:32.000000   02               414661
3  2011-07-29 03:26:51.000000   03               279211
4  2011-07-29 04:18:58.000000   04               225495

In [20]:
py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'})}, filename='complaints per hour')

In [21]:
df = pd.read_sql_query('SELECT CreatedDate, '
                               'strftime(\'%H\', CreatedDate) as `hour`,  '
                               'count(*) as `Complaints per Hour`'
                       'FROM data '
                       'WHERE ComplaintType IN ("Noise", '
                                               '"Noise - Street/Sidewalk", '
                                               '"Noise - Commercial", '
                                               '"Noise - Vehicle", '
                                               '"Noise - Park", '
                                               '"Noise - House of Worship", '
                                               '"Noise - Helicopter", '
                                               '"Collection Truck Noise") '
                       'GROUP BY hour', disk_engine)

display(df.head(n=2))

py.iplot({
    'data': [Bar(x=df['hour'], y=df['Complaints per Hour'])],
    'layout': Layout(xaxis={'title': 'Hour in Day'},
                     yaxis={'title': 'Number of Complaints'},
                     title='Number of Noise Complaints in NYC by Hour in Day'
                    )}, filename='noise complaints per hour')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



CreatedDate hour  Complaints per Hour
0  2011-07-30 00:59:25.000000   00               230224
1  2011-07-30 01:25:58.000000   01               169156

In [22]:
complaint_traces = {} # Each series in the graph will represent a complaint
complaint_traces['Other'] = {}

for hour in range(1, 24):
    hour_str = '0'+str(hour) if hour < 10 else str(hour)
    df = pd.read_sql_query('SELECT  CreatedDate, '
                                   'ComplaintType ,'
                                   'strftime(\'%H\', CreatedDate) as `hour`,  '
                                   'COUNT(*) as num_complaints '
                           'FROM data '
                           'WHERE hour = "{}" '
                           'GROUP BY ComplaintType '
                           'ORDER BY -num_complaints'.format(hour_str), disk_engine)
    
    complaint_traces['Other'][hour] = sum(df.num_complaints)
    
    # Grab the 7 most common complaints for that hour
    for i in range(7):
        complaint = df.get_value(i, 'ComplaintType')
        count = df.get_value(i, 'num_complaints')
        complaint_traces['Other'][hour] -= count
        if complaint in complaint_traces:
            complaint_traces[complaint][hour] = count
        else:
            complaint_traces[complaint] = {hour: count}

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`DataFrame.get_value` defaulting to pandas implementation.

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\pandas\base.py:218: FutureWarning:

get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



In [23]:
traces = []
x_a = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
for complaint in complaint_traces:
    traces.append({
        'x': x_a,
        'y': [complaint_traces[complaint].get(i, None) for i in range(25)],
        'name': complaint,
        'type': 'bar'
    })

py.iplot({
    'data': traces, 
    'layout': {
        'barmode': 'stack',
        'xaxis': {'title': 'Hour in Day'},
        'yaxis': {'title': 'Number of Complaints'},
        'title': 'The 7 Most Common 311 Complaints by Hour in a Day'
    }}, filename='most common complaints by hour')

In [24]:
minutes = 15
seconds = 15*60

df = pd.read_sql_query('SELECT CreatedDate, '
                               'datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval '
                       'FROM data '
                       'LIMIT 10 '.format(seconds=seconds), disk_engine)

display(df.head())

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



CreatedDate             interval
0  2011-07-29 19:51:03.000000  2011-07-29 19:45:00
1  2011-07-29 20:34:33.000000  2011-07-29 20:30:00
2  2011-07-29 22:20:18.000000  2011-07-29 22:15:00
3  2011-07-29 22:35:13.000000  2011-07-29 22:30:00
4  2011-07-29 23:46:33.000000  2011-07-29 23:45:00

In [25]:
minutes = 15
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval '
                       'LIMIT 500'.format(seconds=seconds), disk_engine)

display(df.head())
display(df.tail())

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



interval  Complaints / interval
0  2010-01-01 00:00:00                   2558
1  2010-01-01 00:15:00                     28
2  2010-01-01 00:30:00                     24
3  2010-01-01 00:45:00                     42
4  2010-01-01 01:00:00                     52

interval  Complaints / interval
495  2010-01-06 05:00:00                      6
496  2010-01-06 05:15:00                      2
497  2010-01-06 05:30:00                     16
498  2010-01-06 05:45:00                      6
499  2010-01-06 06:00:00                      6

In [26]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per 15 Minutes'
        }
}, filename='complaints per 15 minutes')

In [27]:
hours = 24
minutes = hours*60
seconds = minutes*60

df = pd.read_sql_query('SELECT datetime(('
                                   'strftime(\'%s\', CreatedDate) / {seconds}) * {seconds}, \'unixepoch\') interval ,'
                               'COUNT(*) as "Complaints / interval"'
                       'FROM data '
                       'GROUP BY interval '
                       'ORDER BY interval'.format(seconds=seconds), disk_engine)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\modin\error_message.py:32: UserWarning:

`read_sql_query` defaulting to pandas implementation.



In [28]:
py.iplot(
    {
        'data': [{
            'x': df.interval,
            'y': df['Complaints / interval'],
            'type': 'bar'
        }],
        'layout': {
            'title': 'Number of 311 Complaints per Day'
        }
}, filename='complaints per day')